# Conceptual Design of a Box-Wing Aircraft

In [10]:
from AeronauticConstants import AeronauticConstants as ac
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The five independent equations are:

$$ x_R =  \frac{V_{stall,\ hyp.}^2}{2 T_0 x_{FR} g} \cdot Q_{MTO} \qquad \text{(Take-Off)} $$

$$ \frac{1}{2} \rho_{0} V_{stall,\ hyp.} ^ 2 S_w C_{L,\ hyp} = Q_{MTO} \qquad \text{(Lift-Off)} $$

$$ x_F = \frac{(V_{stall,\ hyp}^{Land})^2}{2 \bar{\bar a}} \qquad \text{(Braking)} $$

Note: \(V_{stall,\ hyp}^{Land} \ne  V_{stall,\ hyp}\) as they are defined at two different weight conditions (\(Q_{MTO}\) and \(Q_{MTO} - Q_f\)).

$$ (V_{stall,\ hyp}^{Land})^2 = \frac{Q_{MTO} (1 - k)}{\frac{1}{2} \rho_0 S_w C_{L,\ hyp}} \cdot \frac{1}{\bar{\bar a}} \qquad (\text{Landing}) $$

$$ Q_{MTO} = Q_{OE} + Q_{f} + Q_{PL} \qquad (\text{Weights}) $$

Some of the parameters involved are actually from other assumptions or equations (which should increase their number to more than five); however, they are not included here, as these equations relate to the operative aspects of the aircraft and not to its setting (e.g., the autonomy depends on the engine SFC, which is fixed once the engine is fixed; however, the landing distance imposes an external condition on the aircraft setting, which is determined by the fact that the airports are of a certain size).

In [15]:
x_TO = (1300 + 1800) / 2 # [m] Balanced Take-Off Length: from full stop to virtual vertical obstalce at 35 ft (1300 m < x_TO < 1800 m)

z_CR = 20e3 * ac.FT_TO_M # [m] Cruise altitude

x_LD = (1000 + 1500) / 2 # [m] Balanced Landing Length: from V_land = 115 kts to full stop (1000 m < x_TO < 1500 m)

V_land = 115 * ac.KTS_TO_MPS # [m/s]

## Payload weight requirement

In [ ]:
n_pax = 90 # [-] Number of passengers (80 < n_pax < 100)

Q_per_pax = 95 # [kg] mass per passenger

Q_pax = Q_per_pax * n_pax 

# ------ 

n_crew = 4 # [-] Number of company crew members

Q_crew = Q_per_pax * n_crew # [kg] 

# ------

Q_per_seat = 15 # [kg] # mass per seat 

Q_seats = Q_per_seat * n_pax # [kg]

# ------

Q_liq_0 = 200 # [kg]

Q_liq_add_per_pax = 1.9 # [kg]

Q_liq = Q_liq_0 + Q_liq_add_per_pax * n_pax # [kg]

# ------

V_per_lugg = 5 * (ac.FT_TO_M) ** 3 # [m^3]

V_lugg = V_per_lugg * n_pax # [m^3]

# ------

Q_pay = Q_pax + Q_crew + Q_seats + Q_liq

print(f"The total mass of payload to be carried is: Q_pay = {Q_pay} kg")

The total mass of payload to be carried is: Q_pay = 10651.0 kg



From CS 25.1 Applicability:

(a) These Certification Specifications are applicable to turbine powered Large Aeroplanes.

From wikipedia:

"This certification procedure applies to large, turbine-powered aircraft, with max take-off weight more than 5,700kg (CS 25.1)"

So we are in this Aircraft Category

## Autonomy range calculation

In [14]:
Q_MTO = 46000 # [kg] Maximum Take-Off Mass

Q_f_non_usable = 0.003 # [-] Residual ratio of Q_f 

x_range = 1000 * ac.NM_TO_KM * 1000 # [m] Required range

SFC = 1 # [-] Specific Fuel Consumption

E_CR =  1 # [-] Cruise efficiency (lift-over-drag ratio)

V_CR = 1 # [m/s] Cruise velocity

k =  1 - np.exp( - SFC * x_range / (E_CR * V_CR))# [-] Fuel-to-MTO ratio: k = Q_f / Q_MTO

Q_f = k * Q_MTO